In [51]:
import pandas as pd

In [52]:
df = pd.read_csv('../MNH TTO Data/MNH TTO Data - MERGED.csv')

In [53]:
df

,Date,Bed no.,Case no.,Can close,No. of drugs,Time TTO received (24 hr format) (eg 1340),Timing of Packing (24 hr format / PBH) (eg 1340 / PBH),Timing of Dispensing (24 hr format),Comments (for delayed cases) (eg. Pt sleeping etc),Time Taken To Pack (mins),...,Timing of Dispensing,20 min and below,Between 20 to 30 min,30min and below,More than 30 min,Weekday,Weekday Count,Weekend,Weekend Count,NOT Ward 9
0,01-01-21,905,4020095917,839,4,828,pbh,840,NaN,PBH,...,8:40 AM,1,0,1,0,TRUE,1,FALSE,0,0
1,01-01-21,1202,4020097017,934,8,904,850,NaN,wait for 2nd rx,##############################################...,...,#VALUE!,0,0,0,0,TRUE,1,FALSE,0,1
2,01-01-21,1224,4020095632,934,1,901,905,920,NaN,0:04,...,9:20 AM,1,0,1,0,TRUE,1,FALSE,0,1
3,01-01-21,818,4020097033,934,1,900,901,915,NaN,0:01,...,9:15 AM,1,0,1,0,TRUE,1,FALSE,0,1
4,01-01-21,803,4020097026,1029,1,908,915,1027,"can close, added on metformin and glipizide. 1...",0:07,...,10:27 AM,0,0,0,1,TRUE,1,FALSE,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15517,31-12-21,824,4021107656,1311,1,1300,1300,1310,add on TTO,0:00,...,1:10 PM,1,0,1,0,TRUE,1,FALSE,0,1
15518,31-12-21,1205,4021107518,1351,1,1335,pbh,1347,NaN,PBH,...,1:47 PM,1,0,1,0,TRUE,1,FALSE,0,1
15519,31-12-21,1131,4021107644,1426,3,1408,1417,1434,NaN,0:09,...,2:34 PM,0,1,1,0,TRUE,1,FALSE,0,1
15520,31-12-21,622,4021108689,1446,2,1359,1409,1441,1431 pt in bathroom,0:10,...,2:41 PM,0,0,0,1,TRUE,1,FALSE,0,1


In [54]:
df.drop(['Can close', 'Time TTO received (24 hr format) (eg 1340)', 'Timing of Packing (24 hr format / PBH) (eg 1340 / PBH)', 'Timing of Dispensing (24 hr format)'], axis=1, inplace=True)

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15522 entries, 0 to 15521
Data columns (total 19 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   Date                                                15522 non-null  object
 1   Bed no.                                             15512 non-null  object
 2   Case no.                                            15516 non-null  object
 3   No. of drugs                                        15520 non-null  object
 4   Comments (for delayed cases) (eg. Pt sleeping etc)  3862 non-null   object
 5   Time Taken To Pack (mins)                           15521 non-null  object
 6   Time Taken to Reach Pt (mins)                       15522 non-null  object
 7   Time TTO received                                   15522 non-null  object
 8   Timing of packing                                   15522 non-null  object
 9   Timing

In [58]:
df.drop(df.iloc[:, 10:19], inplace = True, axis = 1)

# Handle missing values

In [59]:
df.notnull().sum()

Date                                                  15522
Bed no.                                               15512
Case no.                                              15516
No. of drugs                                          15520
Comments (for delayed cases) (eg. Pt sleeping etc)     3862
Time Taken To Pack (mins)                             15521
Time Taken to Reach Pt (mins)                         15522
Time TTO received                                     15522
Timing of packing                                     15522
Timing of Dispensing                                  15522
dtype: int64

In [60]:
df.isnull().sum()

Date                                                      0
Bed no.                                                  10
Case no.                                                  6
No. of drugs                                              2
Comments (for delayed cases) (eg. Pt sleeping etc)    11660
Time Taken To Pack (mins)                                 1
Time Taken to Reach Pt (mins)                             0
Time TTO received                                         0
Timing of packing                                         0
Timing of Dispensing                                      0
dtype: int64

In [61]:
#drop rows with missing bed number, case number, no. of drugs
#nan_values = df[df['Time TTO received (24 hr format) (eg 1340)'].isna()]
#nan_values
#null_data = df[df.isnull().any(axis=1)]
#null_data
df = df.dropna(subset=['Bed no.', 'Case no.', 'Time Taken To Pack (mins)'])
df.isnull().sum()

#no need to handle missing values for comments since they can leave comments or not

Date                                                      0
Bed no.                                                   0
Case no.                                                  0
No. of drugs                                              0
Comments (for delayed cases) (eg. Pt sleeping etc)    11649
Time Taken To Pack (mins)                                 0
Time Taken to Reach Pt (mins)                             0
Time TTO received                                         0
Timing of packing                                         0
Timing of Dispensing                                      0
dtype: int64

In [62]:
df.head()

,Date,Bed no.,Case no.,No. of drugs,Comments (for delayed cases) (eg. Pt sleeping etc),Time Taken To Pack (mins),Time Taken to Reach Pt (mins),Time TTO received,Timing of packing,Timing of Dispensing
0,01-01-21,905,4020095917,4,NaN,PBH,0:12,8:28 AM,PBH,8:40 AM
1,01-01-21,1202,4020097017,8,wait for 2nd rx,##############################################...,blank,9:04 AM,8:50 AM,#VALUE!
2,01-01-21,1224,4020095632,1,NaN,0:04,0:19,9:01 AM,9:05 AM,9:20 AM
3,01-01-21,818,4020097033,1,NaN,0:01,0:15,9:00 AM,9:01 AM,9:15 AM
4,01-01-21,803,4020097026,1,"can close, added on metformin and glipizide. 1...",0:07,1:19,9:08 AM,9:15 AM,10:27 AM


# Remove rows with special characters

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15507 entries, 0 to 15521
Data columns (total 10 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   Date                                                15507 non-null  object
 1   Bed no.                                             15507 non-null  object
 2   Case no.                                            15507 non-null  object
 3   No. of drugs                                        15507 non-null  object
 4   Comments (for delayed cases) (eg. Pt sleeping etc)  3858 non-null   object
 5   Time Taken To Pack (mins)                           15507 non-null  object
 6   Time Taken to Reach Pt (mins)                       15507 non-null  object
 7   Time TTO received                                   15507 non-null  object
 8   Timing of packing                                   15507 non-null  object
 9   Timing

In [73]:
# drop rows with ######### value in Time Taken to Reach Pt (mins) metric
#df[df.iloc[:, 5:6].contains(r'[^0-9a-zA-Z]').index]

df = df.loc[df['Time Taken To Pack (mins)'].str.contains(r'#') == False]
df = df.loc[df['Time Taken to Reach Pt (mins)'].str.contains(r'#') == False]
df

,Date,Bed no.,Case no.,No. of drugs,Comments (for delayed cases) (eg. Pt sleeping etc),Time Taken To Pack (mins),Time Taken to Reach Pt (mins),Time TTO received,Timing of packing,Timing of Dispensing
0,01-01-21,905,4020095917,4,NaN,PBH,0:12,8:28 AM,PBH,8:40 AM
2,01-01-21,1224,4020095632,1,NaN,0:04,0:19,9:01 AM,9:05 AM,9:20 AM
3,01-01-21,818,4020097033,1,NaN,0:01,0:15,9:00 AM,9:01 AM,9:15 AM
4,01-01-21,803,4020097026,1,"can close, added on metformin and glipizide. 1...",0:07,1:19,9:08 AM,9:15 AM,10:27 AM
5,01-01-21,1229,4020096397,3,NaN,0:05,0:45,9:20 AM,9:25 AM,10:05 AM
...,...,...,...,...,...,...,...,...,...,...
15517,31-12-21,824,4021107656,1,add on TTO,0:00,0:10,1:00 PM,1:00 PM,1:10 PM
15518,31-12-21,1205,4021107518,1,NaN,PBH,0:12,1:35 PM,PBH,1:47 PM
15519,31-12-21,1131,4021107644,3,NaN,0:09,0:26,2:08 PM,2:17 PM,2:34 PM
15520,31-12-21,622,4021108689,2,1431 pt in bathroom,0:10,0:42,1:59 PM,2:09 PM,2:41 PM


# Remove rows with "blank" values

In [76]:
df = df.loc[df['Time Taken To Pack (mins)'].str.contains(r'blank') == False]

In [77]:
df = df.loc[df['Time Taken to Reach Pt (mins)'].str.contains(r'blank') == False]

In [78]:
df

,Date,Bed no.,Case no.,No. of drugs,Comments (for delayed cases) (eg. Pt sleeping etc),Time Taken To Pack (mins),Time Taken to Reach Pt (mins),Time TTO received,Timing of packing,Timing of Dispensing
0,01-01-21,905,4020095917,4,NaN,PBH,0:12,8:28 AM,PBH,8:40 AM
2,01-01-21,1224,4020095632,1,NaN,0:04,0:19,9:01 AM,9:05 AM,9:20 AM
3,01-01-21,818,4020097033,1,NaN,0:01,0:15,9:00 AM,9:01 AM,9:15 AM
4,01-01-21,803,4020097026,1,"can close, added on metformin and glipizide. 1...",0:07,1:19,9:08 AM,9:15 AM,10:27 AM
5,01-01-21,1229,4020096397,3,NaN,0:05,0:45,9:20 AM,9:25 AM,10:05 AM
...,...,...,...,...,...,...,...,...,...,...
15517,31-12-21,824,4021107656,1,add on TTO,0:00,0:10,1:00 PM,1:00 PM,1:10 PM
15518,31-12-21,1205,4021107518,1,NaN,PBH,0:12,1:35 PM,PBH,1:47 PM
15519,31-12-21,1131,4021107644,3,NaN,0:09,0:26,2:08 PM,2:17 PM,2:34 PM
15520,31-12-21,622,4021108689,2,1431 pt in bathroom,0:10,0:42,1:59 PM,2:09 PM,2:41 PM


In [79]:
df.to_csv("MNH TTO TAT (cleaned).csv", index = None, header=True)

# EDA

In [82]:
df = pd.read_csv('MNH TTO TAT (cleaned).csv')

In [83]:
df.describe()

,Date,Bed no.,Case no.,No. of drugs,Comments (for delayed cases) (eg. Pt sleeping etc),Time Taken To Pack (mins),Time Taken to Reach Pt (mins),Time TTO received,Timing of packing,Timing of Dispensing
count,15320,15320,15320,15320,3796,15320,15320,15320,15320,15320
unique,392,395,14712,29,2294,180,338,865,809,774
top,12-03-21,626,4021052393,1,paeds relabel,PBH,0:25,9:00 AM,PBH,9:30 AM
freq,106,117,4,3599,161,3819,394,86,3828,190
